# eda018
gnss_logのセンサ周りの特徴量　スムージング追加

In [1]:
import os
import pandas as pd
import numpy as np
import ipynb_path
import matplotlib.pyplot as plt
from matplotlib_venn import venn2, venn2_circles
import seaborn as sns
import plotly
import plotly.express as px
%matplotlib inline
pd.set_option('display.max_rows', 200)
from math import * 
import warnings
warnings.simplefilter('ignore')

In [2]:
def get_nb_name():
    nb_path = ipynb_path.get()
    nb_name = nb_path.rsplit('/',1)[1].replace('.ipynb','')
    return nb_name

In [3]:
def visualize_trafic(df, center, zoom=9):
    fig = px.scatter_mapbox(df,
                            
                            # Here, plotly gets, (x,y) coordinates
                            lat="latDeg",
                            lon="lngDeg",
                            
                            #Here, plotly detects color of series
                            color="phoneName",
                            labels="phoneName",
                            
                            zoom=zoom,
                            center=center,
                            height=1000,
                            width=2000)
    fig.update_layout(mapbox_style='stamen-terrain')
    fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
    fig.update_layout(title_text="GPS trafic")
    fig.show()

In [4]:
def visualize_collection(df, collection):
    target_df = df[df['collectionName']==collection].copy()
    lat_center = target_df['latDeg'].mean()
    lng_center = target_df['lngDeg'].mean()
    center = {"lat":lat_center, "lon":lng_center}
    
    visualize_trafic(target_df, center)

In [5]:
def calc_haversine(lat1, lon1, lat2, lon2):
    """Calculates the great circle distance between two points
    on the earth. Inputs are array-like and specified in decimal degrees.
    """
    RADIUS = 6_367_000
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    dist = 2 * RADIUS * np.arcsin(a**0.5)
    return dist

In [6]:
# lowpass filter

from scipy.signal import butter, lfilter

def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y

order = 3
fs = 50.0
cutoff = 2.5

In [7]:
# Offset correction
# refarence https://github.com/J-ROCKET-BOY/SS-Fitting

def SS_fit(data) : 

    x = data[:,[0]]
    y = data[:,[1]]
    z = data[:,[2]]

    data_len = len(x)
    
    x2 = np.power(x,2)
    y2 = np.power(y,2)
    z2 = np.power(z,2)

    r1 = -x*(x2+y2+z2)
    r2= -y*(x2+y2+z2)
    r3 = -z*(x2+y2+z2)
    r4 = -(x2+y2+z2)

    left = np.array([[np.sum(x2),np.sum(x*y),np.sum(x*z),np.sum(x)],
                     [np.sum(x*y),np.sum(y2),np.sum(y*z),np.sum(y)],
                     [np.sum(x*z),np.sum(y*z),np.sum(z2),np.sum(z)],
                     [np.sum(x), np.sum(y), np.sum(z), data_len]])
    
    right = np.array([np.sum(r1),
                      np.sum(r2),
                      np.sum(r3),
                      np.sum(r4)])
    
    si = np.dot(np.linalg.inv(left),right)

    x0 = (-1/2)* si[0]
    y0 = (-1/2)* si[1]
    z0 = (-1/2)* si[2]
    
    return np.array([x0,y0,z0])

In [8]:
# Vincenty's formulae
# refarence https://qiita.com/r-fuji/items/99ca549b963cedc106ab

def vincenty_inverse(lat1, lon1, lat2, lon2):

    # Not advanced
    if isclose(lat1, lat2) and isclose(lon1, lon2):
        return False
    
    # WGS84
    a = 6378137.0
    ƒ = 1 / 298.257223563
    b = (1 - ƒ) * a

    lat_1 = atan((1 - ƒ) * tan(radians(lat1)))
    lat_2 = atan((1 - ƒ) * tan(radians(lat2)))
    
    lon_diff = radians(lon2) - radians(lon1)
    λ = lon_diff

    for i in range(1000):
        sinλ = sin(λ)
        cosλ = cos(λ)
        sinσ = sqrt((cos(lat_2) * sinλ) ** 2 + (cos(lat_1) * sin(lat_2) - sin(lat_1) * cos(lat_2) * cosλ) ** 2)
        cosσ = sin(lat_1) * sin(lat_2) + cos(lat_1) * cos(lat_2) * cosλ
        σ = atan2(sinσ, cosσ)
        sinα = cos(lat_1) * cos(lat_2) * sinλ / sinσ
        cos2α = 1 - sinα ** 2
        cos2σm = cosσ - 2 * sin(lat_1) * sin(lat_2) / cos2α
        C = ƒ / 16 * cos2α * (4 + ƒ * (4 - 3 * cos2α))
        λʹ = λ
        λ = lon_diff + (1 - C) * ƒ * sinα * (σ + C * sinσ * (cos2σm + C * cosσ * (-1 + 2 * cos2σm ** 2)))
        
        if abs(λ - λʹ) <= 1e-12:
            break
    else:
        return None

    α = atan2(cos(lat_2) * sinλ, cos(lat_1) * sin(lat_2) - sin(lat_1) * cos(lat_2) * cosλ)

    if α < 0:
        α = α + pi * 2

    return degrees(α)

In [9]:
def calc3(row):
    deg = - degrees(atan2(-1*row['calc2'],row['calc1']))
    if deg < 0:
        deg += 360
    return deg 

In [10]:
# directory setting
nb_name = get_nb_name()
INPUT = '../input/google-smartphone-decimeter-challenge'
OUTPUT = '../output/' + nb_name
os.makedirs(OUTPUT, exist_ok=True)

# 既存

In [11]:
accel_train = pd.read_csv(INPUT + '/prep/gnss/train/UncalAccel.csv')
gyro_train = pd.read_csv(INPUT + '/prep/gnss/train/UncalGyro.csv')
mag_train = pd.read_csv(INPUT + '/prep/gnss/train/UncalMag.csv')
train = pd.read_csv(INPUT + '/' + 'baseline_locations_train.csv')
ground_truth = pd.read_csv(INPUT + '/prep/ground_truth_train.csv')

ori_train = pd.read_csv(INPUT + '/prep/gnss/train/OrientationDeg.csv')

ground_truth = ground_truth.rename(columns={'latDeg':'latDeg_gt', 'lngDeg':'lngDeg_gt'})
gt = ground_truth[['collectionName', 'phoneName', 'millisSinceGpsEpoch', 'latDeg_gt', 'lngDeg_gt', 'speedMps', 'courseDegree']].copy()
train = train.merge(ground_truth, on=['collectionName', 'phoneName', 'millisSinceGpsEpoch'], how='inner')

In [12]:
def add_sensor_features(df, accel, gyro, mag):
    # phoneを追加
    df['phone'] = df['collectionName'] + '_' + df['phoneName']
    accel['phone'] = accel['collectionName'] + '_' + accel['phoneName']
    gyro['phone'] = gyro['collectionName'] + '_' + gyro['phoneName']
    mag['phone'] = mag['collectionName'] + '_' + mag['phoneName']
    
    # phonenameをラベルエンコーディング
    phoneName_map = {'Pixel4':1, 'Pixel4XLModded':2, 'Pixel4XL':3, 'Mi8':4, 'Pixel4Modded':5, 'Pixel5':6, 'SamsungS20Ultra':7}
    df['phoneName_le'] = df['phoneName'].map(phoneName_map)
    
    # utc -> gps
    accel['millisSinceGpsEpoch'] = accel['utcTimeMillis'] - 315964800000 + 18000
    gyro['millisSinceGpsEpoch'] = gyro['utcTimeMillis'] - 315964800000 + 18000
    mag['millisSinceGpsEpoch'] = mag['utcTimeMillis'] - 315964800000 + 18000
    
    # resampling追加
    df['secondSinceGpsEpoch'] = df['millisSinceGpsEpoch'] // 1000
    accel['secondSinceGpsEpoch'] = accel['millisSinceGpsEpoch'] // 1000
    gyro['secondSinceGpsEpoch'] = gyro['millisSinceGpsEpoch'] // 1000
    mag['secondSinceGpsEpoch'] = mag['millisSinceGpsEpoch'] // 1000
    
    # clipping
    accel[['UncalAccelXMps2', 'UncalAccelYMps2', 'UncalAccelZMps2']] = accel.groupby('phone')['UncalAccelXMps2', 'UncalAccelYMps2', 'UncalAccelZMps2'].transform(lambda x: x.clip(x.quantile(0.001), x.quantile(0.999)))
    gyro[['UncalGyroXRadPerSec', 'UncalGyroYRadPerSec', 'UncalGyroZRadPerSec']] = gyro.groupby('phone')['UncalGyroXRadPerSec', 'UncalGyroYRadPerSec', 'UncalGyroZRadPerSec'].transform(lambda x: x.clip(x.quantile(0.001), x.quantile(0.999)))
    mag[['UncalMagXMicroT', 'UncalMagYMicroT', 'UncalMagZMicroT']] = mag.groupby('phone')['UncalMagXMicroT', 'UncalMagYMicroT', 'UncalMagZMicroT'].transform(lambda x: x.clip(x.quantile(0.001), x.quantile(0.999)))
    
    for c in ['UncalAccelXMps2', 'UncalAccelYMps2', 'UncalAccelZMps2']:
        accel[c] = accel.groupby('phone')[c].rolling(1000, center=True, min_periods=1).mean().values
    for c in ['UncalGyroXRadPerSec', 'UncalGyroYRadPerSec', 'UncalGyroZRadPerSec']:
        gyro[c] = gyro.groupby('phone')[c].rolling(1000, center=True, min_periods=1).mean().values
    for c in ['UncalMagXMicroT', 'UncalMagYMicroT', 'UncalMagZMicroT']:
        mag[c] = mag.groupby('phone')[c].rolling(1000, center=True, min_periods=1).mean().values
    
    accel = accel.groupby(['phone', 'secondSinceGpsEpoch'])['UncalAccelXMps2', 'UncalAccelYMps2', 'UncalAccelZMps2'].agg(['mean', 'std']).reset_index()
    accel.columns = ['phone', 'secondSinceGpsEpoch', 'UncalAccelXMps2_mean', 'UncalAccelXMps2_std', 'UncalAccelYMps2_mean', 'UncalAccelYMps2_std', 'UncalAccelZMps2_mean', 'UncalAccelZMps2_std']
    gyro = gyro.groupby(['phone', 'secondSinceGpsEpoch'])['UncalGyroXRadPerSec', 'UncalGyroYRadPerSec', 'UncalGyroZRadPerSec'].agg(['mean', 'std']).reset_index()
    gyro.columns = ['phone', 'secondSinceGpsEpoch', 'UncalGyroXRadPerSec_mean', 'UncalGyroXRadPerSec_std', 'UncalGyroYRadPerSec_mean', 'UncalGyroYRadPerSec_std', 'UncalGyroZRadPerSec_mean', 'UncalGyroZRadPerSec_std' ]
    mag = mag.groupby(['phone', 'secondSinceGpsEpoch'])['UncalMagXMicroT', 'UncalMagYMicroT', 'UncalMagZMicroT'].agg(['mean', 'std']).reset_index()
    mag.columns = ['phone', 'secondSinceGpsEpoch', 'UncalMagXMicroT_mean', 'UncalMagXMicroT_std', 'UncalMagYMicroT_mean', 'UncalMagYMicroT_std', 'UncalMagZMicroT_mean', 'UncalMagZMicroT_std']
    
    df = df.merge(accel, on=['phone', 'secondSinceGpsEpoch'], how='left')
    df = df.merge(gyro, on=['phone', 'secondSinceGpsEpoch'], how='left')
    df = df.merge(mag, on=['phone', 'secondSinceGpsEpoch'], how='left')   
    
    df.drop(['secondSinceGpsEpoch'], axis=1, inplace=True)
    
    return df

In [13]:
accel_train

,utcTimeMillis,elapsedRealtimeNanos,UncalAccelXMps2,UncalAccelYMps2,UncalAccelZMps2,collectionName,phoneName,BiasXMps2,BiasYMps2,BiasZMps2
0,1590096145767,388949302725,-2.127655,10.233923,-0.053478,2020-05-21-US-MTV-2,Pixel4,NaN,NaN,NaN
1,1590096145769,388951731996,-2.427884,10.108892,-0.152563,2020-05-21-US-MTV-2,Pixel4,NaN,NaN,NaN
2,1590096145772,388954188610,-2.649440,10.083163,-0.206106,2020-05-21-US-MTV-2,Pixel4,NaN,NaN,NaN
3,1590096145774,388956631579,-2.760863,10.055699,-0.271014,2020-05-21-US-MTV-2,Pixel4,NaN,NaN,NaN
4,1590096145777,388959060850,-2.775203,10.070100,-0.285293,2020-05-21-US-MTV-2,Pixel4,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
20523367,1590796844791,2214597316626,0.133604,10.150373,1.042262,2020-05-29-US-MTV-2,Pixel4XL,NaN,NaN,NaN
20523368,1590796844800,2214606996262,-0.009243,10.186256,1.146229,2020-05-29-US-MTV-2,Pixel4XL,NaN,NaN,NaN
20523369,1590796844810,2214616675897,-0.114211,10.055800,1.365687,2020-05-29-US-MTV-2,Pixel4XL,NaN,NaN,NaN
20523370,1590796844820,2214626355584,-0.131431,9.785808,1.162168,2020-05-29-US-MTV-2,Pixel4XL,NaN,NaN,NaN


In [14]:
org = add_sensor_features(train, accel_train, gyro_train, mag_train)

In [16]:
def viz(df, cols, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    cols_n = len(cols)
    phones = df['phone'].unique()
    
    for phone in phones:
        df_tmp = df[df['phone']==phone].copy()
        
        fig, axes = plt.subplots(figsize=(5*cols_n, 20), nrows=cols_n,sharex=True)
        for i,c in enumerate(cols):
            axes[i].plot(df_tmp['millisSinceGpsEpoch'], df_tmp[c], label=c)
            axes[i].legend(loc='upper right')
            axes[i].grid(color='g', linestyle=':', linewidth=0.3)
        fig.suptitle(phone, fontsize=16)
        fig.savefig(output_dir + '/' + phone + '.png')
        plt.close()

In [17]:
viz(org, 
    cols=['speedMps', 'courseDegree', 'UncalAccelXMps2_mean', 'UncalAccelXMps2_std', 'UncalAccelYMps2_mean', 'UncalAccelYMps2_std', 'UncalAccelZMps2_mean', 'UncalAccelZMps2_std'],
    output_dir = OUTPUT + '/gnss_ts/train/accel')

In [18]:
viz(org, 
    cols=['speedMps', 'UncalGyroXRadPerSec_mean', 'UncalGyroXRadPerSec_std', 'UncalGyroYRadPerSec_mean', 'UncalGyroYRadPerSec_std', 'UncalGyroZRadPerSec_mean', 'UncalGyroZRadPerSec_std'],
    output_dir = OUTPUT + '/gnss_ts/train/gyro')

In [19]:
viz(org, 
    cols=['speedMps', 'UncalMagXMicroT_mean', 'UncalMagXMicroT_std', 'UncalMagYMicroT_mean', 'UncalMagYMicroT_std', 'UncalMagZMicroT_mean', 'UncalMagZMicroT_std'],
    output_dir = OUTPUT + '/gnss_ts/train/mag')

# 角度算出の公開ノートブック

In [20]:
from pathlib import Path
data_dir = Path('../input/google-smartphone-decimeter-challenge')
logg_files = (data_dir / 'train').rglob('*GnssLog.txt')
logg_paths = [str(p) for p in logg_files]

In [21]:
def gnss_log_to_dataframes(path):
    print('Loading ' + path, flush=True)
    gnss_section_names = {'Raw','UncalAccel', 'UncalGyro', 'UncalMag', 'Fix', 'Status', 'OrientationDeg'}
    with open(path) as f_open:
        datalines = f_open.readlines()

    datas = {k: [] for k in gnss_section_names}
    gnss_map = {k: [] for k in gnss_section_names}
    for dataline in datalines:
        is_header = dataline.startswith('#')
        dataline = dataline.strip('#').strip().split(',')
        # skip over notes, version numbers, etc
        if is_header and dataline[0] in gnss_section_names:
            try:
                gnss_map[dataline[0]] = dataline[1:]
            except:
                pass
        elif not is_header:
            try:
                datas[dataline[0]].append(dataline[1:])
            except:
                pass
    results = dict()
    for k, v in datas.items():
        results[k] = pd.DataFrame(v, columns=gnss_map[k])
    # pandas doesn't properly infer types from these lists by default
    for k, df in results.items():
        for col in df.columns:
            if col == 'CodeType':
                continue
            try:
                results[k][col] = pd.to_numeric(results[k][col])
            except:
                pass
    return results


In [22]:
pub = pd.DataFrame()
for n, logg_path in enumerate(logg_paths):
                      
#     if n > 3:
#         break
    path = logg_path
    gt_path = logg_path.rsplit('/',1)[0]+"/ground_truth.csv"

    gt_df = pd.read_csv(gt_path)
    sample = gnss_log_to_dataframes(str(path))

    acce_df = sample["UncalAccel"]
    mag_df = sample["UncalMag"]

    if not len(acce_df):
        continue
    
    acce_df["millisSinceGpsEpoch"] = acce_df["utcTimeMillis"] - 315964800000 + 18000
    mag_df["millisSinceGpsEpoch"] = mag_df["utcTimeMillis"] - 315964800000 + 18000
    
#     acce filtering and smooting
    
    #acce_df["global_x"] = acce_df["UncalAccelZMps2"]
    #acce_df["global_y"] = acce_df["UncalAccelXMps2"]
    #acce_df["global_z"] = acce_df["UncalAccelYMps2"]
    
    acce_df["global_x"] = acce_df["UncalAccelXMps2"]
    acce_df["global_y"] = acce_df["UncalAccelYMps2"]
    acce_df["global_z"] = acce_df["UncalAccelZMps2"]
    
    acce_df["x_f"] = butter_lowpass_filter(acce_df["global_x"], cutoff, fs, order)
    acce_df["y_f"] = butter_lowpass_filter(acce_df["global_y"], cutoff, fs, order)
    acce_df["z_f"] = butter_lowpass_filter(acce_df["global_z"], cutoff, fs, order)
    
    smooth_range = 1000

    acce_df["x_f"] = acce_df["x_f"].rolling(smooth_range, center=True, min_periods=1).mean()
    acce_df["y_f"] = acce_df["y_f"].rolling(smooth_range, center=True, min_periods=1).mean()
    acce_df["z_f"] = acce_df["z_f"].rolling(smooth_range, center=True, min_periods=1).mean()
    
#     magn filtering and smooting , offset correction
    
    mag_df["global_mx"] = mag_df["UncalMagXMicroT"]
    mag_df["global_my"] = mag_df["UncalMagYMicroT"]
    mag_df["global_mz"] = mag_df["UncalMagZMicroT"]
    
    smooth_range = 1000
    
    mag_df["global_mx"] = mag_df["global_mx"].rolling(smooth_range,  min_periods=1).mean()
    mag_df["global_my"] = mag_df["global_my"].rolling(smooth_range,  min_periods=1).mean()
    mag_df["global_mz"] = mag_df["global_mz"].rolling(smooth_range,  min_periods=1).mean()
    
#     merge the value of the one with the closest time 
    
    acce_df["millisSinceGpsEpoch"] = acce_df["millisSinceGpsEpoch"]//1000
    mag_df["millisSinceGpsEpoch"] = mag_df["millisSinceGpsEpoch"]//1000
    gt_df["millisSinceGpsEpoch"] = gt_df["millisSinceGpsEpoch"]//1000
    
    acce_df = pd.merge_asof(acce_df.sort_values('millisSinceGpsEpoch'), 
                           mag_df[["global_mx", "global_my","global_mz","millisSinceGpsEpoch"]].sort_values('millisSinceGpsEpoch') ,on='millisSinceGpsEpoch', direction='nearest')
    
    acce_df = pd.merge_asof(gt_df[["millisSinceGpsEpoch","latDeg","lngDeg"]].sort_values('millisSinceGpsEpoch'), 
                           acce_df[["millisSinceGpsEpoch", "x_f","y_f","z_f","global_mx","global_my","global_mz"]].sort_values('millisSinceGpsEpoch') ,on='millisSinceGpsEpoch', direction='nearest')
    
    acce_df['collectionName'] = logg_path.split('/')[4]
    acce_df['phoneName'] = logg_path.split('/')[5]
    
    pub = pub.append(acce_df)

Loading ../input/google-smartphone-decimeter-challenge/train/2020-05-21-US-MTV-2/Pixel4/Pixel4_GnssLog.txt
Loading ../input/google-smartphone-decimeter-challenge/train/2020-05-21-US-MTV-2/Pixel4XL/Pixel4XL_GnssLog.txt
Loading ../input/google-smartphone-decimeter-challenge/train/2020-06-11-US-MTV-1/Pixel4/Pixel4_GnssLog.txt
Loading ../input/google-smartphone-decimeter-challenge/train/2020-06-11-US-MTV-1/Pixel4XL/Pixel4XL_GnssLog.txt
Loading ../input/google-smartphone-decimeter-challenge/train/2020-06-05-US-MTV-2/Pixel4/Pixel4_GnssLog.txt
Loading ../input/google-smartphone-decimeter-challenge/train/2020-06-05-US-MTV-2/Pixel4XL/Pixel4XL_GnssLog.txt
Loading ../input/google-smartphone-decimeter-challenge/train/2020-07-17-US-MTV-1/Mi8/Mi8_GnssLog.txt
Loading ../input/google-smartphone-decimeter-challenge/train/2021-04-22-US-SJC-1/Pixel4/Pixel4_GnssLog.txt
Loading ../input/google-smartphone-decimeter-challenge/train/2021-04-22-US-SJC-1/SamsungS20Ultra/SamsungS20Ultra_GnssLog.txt
Loading ../in

In [23]:
org.columns

Index(['collectionName', 'phoneName', 'millisSinceGpsEpoch', 'latDeg',
       'lngDeg', 'heightAboveWgs84EllipsoidM_x', 'phone', 'latDeg_gt',
       'lngDeg_gt', 'heightAboveWgs84EllipsoidM_y', 'timeSinceFirstFixSeconds',
       'hDop', 'vDop', 'speedMps', 'courseDegree', 'phoneName_le',
       'UncalAccelXMps2_mean', 'UncalAccelXMps2_std', 'UncalAccelYMps2_mean',
       'UncalAccelYMps2_std', 'UncalAccelZMps2_mean', 'UncalAccelZMps2_std',
       'UncalGyroXRadPerSec_mean', 'UncalGyroXRadPerSec_std',
       'UncalGyroYRadPerSec_mean', 'UncalGyroYRadPerSec_std',
       'UncalGyroZRadPerSec_mean', 'UncalGyroZRadPerSec_std',
       'UncalMagXMicroT_mean', 'UncalMagXMicroT_std', 'UncalMagYMicroT_mean',
       'UncalMagYMicroT_std', 'UncalMagZMicroT_mean', 'UncalMagZMicroT_std'],
      dtype='object')

In [24]:
pub['phone'] = pub['collectionName'] + '_' + pub['phoneName']

In [25]:
pub['millisSinceGpsEpoch'] = pub['millisSinceGpsEpoch'] * 1000

In [26]:
output_dir = OUTPUT + '/compare/accel'
os.makedirs(output_dir, exist_ok=True)

for phone in pub['phone'].unique():

    fig, axes = plt.subplots(figsize=(5*3, 20), nrows=3,sharex=True)
    org_tmp = org[org['phone']==phone].copy()
    pub_tmp = pub[pub['phone']==phone].copy()

    axes[0].plot(org_tmp['millisSinceGpsEpoch'], org_tmp['UncalAccelXMps2_mean'], label='UncalAccelXMps2_mean')
    axes[0].plot(pub_tmp['millisSinceGpsEpoch'], pub_tmp['x_f'], label='global_x')
    axes[0].legend(loc='upper right')
    axes[0].grid(color='g', linestyle=':', linewidth=0.3)

    axes[1].plot(org_tmp['millisSinceGpsEpoch'], org_tmp['UncalAccelYMps2_mean'], label='UncalAccelYMps2_mean')
    axes[1].plot(pub_tmp['millisSinceGpsEpoch'], pub_tmp['y_f'], label='global_y')
    axes[1].legend(loc='upper right')
    axes[1].grid(color='g', linestyle=':', linewidth=0.3)

    axes[2].plot(org_tmp['millisSinceGpsEpoch'], org_tmp['UncalAccelZMps2_mean'], label='UncalAccelZMps2_mean')
    axes[2].plot(pub_tmp['millisSinceGpsEpoch'], pub_tmp['z_f'], label='global_z')
    axes[2].legend(loc='upper right')
    axes[2].grid(color='g', linestyle=':', linewidth=0.3)
    fig.suptitle(phone, fontsize=16)
    fig.savefig(output_dir + '/' + phone + '.png')
    plt.close()

In [27]:
output_dir = OUTPUT + '/compare/mag'
os.makedirs(output_dir, exist_ok=True)

for phone in pub['phone'].unique():

    fig, axes = plt.subplots(figsize=(5*3, 20), nrows=3,sharex=True)
    org_tmp = org[org['phone']==phone].copy()
    pub_tmp = pub[pub['phone']==phone].copy()

    axes[0].plot(org_tmp['millisSinceGpsEpoch'], org_tmp['UncalMagXMicroT_mean'], label='UncalMagXMicroT_mean')
    axes[0].plot(pub_tmp['millisSinceGpsEpoch'], pub_tmp['global_mx'], label='global_x')
    axes[0].legend(loc='upper right')
    axes[0].grid(color='g', linestyle=':', linewidth=0.3)

    axes[1].plot(org_tmp['millisSinceGpsEpoch'], org_tmp['UncalMagYMicroT_mean'], label='UncalMagYMicroT_mean')
    axes[1].plot(pub_tmp['millisSinceGpsEpoch'], pub_tmp['global_my'], label='global_y')
    axes[1].legend(loc='upper right')
    axes[1].grid(color='g', linestyle=':', linewidth=0.3)

    axes[2].plot(org_tmp['millisSinceGpsEpoch'], org_tmp['UncalMagZMicroT_mean'], label='UncalMagZMicroT_mean')
    axes[2].plot(pub_tmp['millisSinceGpsEpoch'], pub_tmp['global_mz'], label='global_z')
    axes[2].legend(loc='upper right')
    axes[2].grid(color='g', linestyle=':', linewidth=0.3)
    fig.suptitle(phone, fontsize=16)
    fig.savefig(output_dir + '/' + phone + '.png')
    plt.close()